In [1]:
import pandas as pd
import numpy as np
import json
import re

%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (15,8)


# pandas max row 
pd.options.display.max_rows = 3000

## **Connecting**

In [9]:
pip install geopandas

In [10]:
pip install folium

In [11]:
import geopandas as gpd
import folium

## Data setting

In [19]:
information_path = '/content/drive/MyDrive/Sejong/매매값_아파트정보.csv'

df = pd.read_csv(information_path)
df['동'] = df['PNU'].str[:-8]
df_dong = df.groupby('동').agg({'거래금액' : 'count','층수' : 'max'})
df_dong = df_dong.reset_index()
df_dong.head()

동  거래금액    층수
0  고운동  3007  29.0
1  금남면   585  15.0
2  나성동   652   8.0
3  다정동   888  35.0
4  대평동   286  30.0

In [21]:
geo_ad = gpd.read_file('/content/drive/MyDrive/Sejong/23.세종시_도로명주소_건물.geojson')
geo_dong = gpd.read_file('/content/drive/MyDrive/Sejong/31.세종시_법정경계(읍면동).geojson')
geo_dong = geo_dong.rename({'EMD_KOR_NM' : '동'},axis = 'columns')
display(geo_ad.head(), geo_dong.head())

BDTYP_CD  ...                                           geometry
0    10299  ...  MULTIPOLYGON (((127.19039 36.49291, 127.19052 ...
1    10299  ...  MULTIPOLYGON (((127.29546 36.42295, 127.29564 ...
2    04401  ...  MULTIPOLYGON (((127.26217 36.47936, 127.26227 ...
3    04001  ...  MULTIPOLYGON (((127.26235 36.47986, 127.26220 ...
4    03999  ...  MULTIPOLYGON (((127.26173 36.48000, 127.26188 ...

[5 rows x 11 columns]

EMD_CD    동                                           geometry
0  36110101  반곡동  MULTIPOLYGON (((127.30833 36.47977, 127.30818 ...
1  36110102  소담동  MULTIPOLYGON (((127.30480 36.48845, 127.30480 ...
2  36110103  보람동  MULTIPOLYGON (((127.29383 36.48508, 127.29381 ...
3  36110104  대평동  MULTIPOLYGON (((127.27215 36.46388, 127.27197 ...
4  36110105  가람동  MULTIPOLYGON (((127.24631 36.46300, 127.24626 ...

In [22]:
# 도담동 법제처
center = [36.506094, 127.259488]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m = folium.Map(location=center, zoom_start=10)

# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=geo_dong,
    data=df_dong,
    columns=('동', '거래금액'),
    key_on='feature.properties.동',
    fill_color='BuPu',
    legend_name='거래금액',
).add_to(m)

# 맵 m을 출력합니다.
m